In [67]:
!pip install transformers datasets evaluate accelerate

In [68]:
# import torch
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder

In [69]:
!pip install parsivar

In [70]:
import pandas as pd
from parsivar import Normalizer
import re

# Read the TSV file into a DataFrame
df = pd.read_csv('train.tsv', sep='\t', header=None, names=['Sentence', 'Label'], encoding='utf-8')
df_test = pd.read_csv('test.tsv', sep='\t', header=None, names=['Sentence', 'Label'], encoding='utf-8')

# Initialize Parsivar normalizer
normalizer = Normalizer()

# Function to perform additional pre-processing steps
def additional_preprocessing(text):
    # Remove English characters
    text = re.sub(r'[a-zA-Z]', '', text)

    # Remove repeated letters more than twice in non-standard Persian words
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)

    # Remove Arabic diacritics
    text = re.sub(r'[\u064B-\u065F\u0670]', '', text)

    # Remove remaining non-Persian characters
    text = re.sub(r'[^آ-ی۰-۹\s]', '', text)

    # Remove hashtag sign while preserving hashtag information
    text = re.sub(r'#(\w+)', r'\1', text)

    # Remove Persian numeric characters
    text = re.sub(r'[۰-۹]', '', text)

    return text

# Apply pre-processing to the 'Sentence' column
df['Sentence'] = df['Sentence'].apply(normalizer.normalize)
df['Sentence'] = df['Sentence'].apply(additional_preprocessing)
df = df.rename(columns={'Sentence': 'text', 'Label': 'label'})

df_test['Sentence'] = df_test['Sentence'].apply(normalizer.normalize)
df_test['Sentence'] = df_test['Sentence'].apply(additional_preprocessing)
df_test = df_test.rename(columns={'Sentence': 'text', 'Label': 'label'})

# Display the DataFrame
print(df)


                                                   text     label
0     خیلی کوچیک هستن و سایزشون بدرد نمیخوره میخوام ...       SAD
1        از صدای پرنده دم دمای صبح متنفرم متنفرم متنفرم      HATE
2      کیفیتش خیلی خوبه با شک خریدم ولی واقعا راضیم ...       SAD
3     چون همش با دوربین ثبتشده ایا میشه اعتراض زد  و...     OTHER
4                     این وضع ب طرز خندهداری گریه داره        SAD
...                                                 ...       ...
6120  مرحوم پیشبینی آبکی زیاد میکرد مرحوم عجب آینده ...  SURPRISE
6121  کلا عین اعتقادات و توئیت زدناتون   در قبال ران...     ANGRY
6122  خب وقتی میگی کسی بیاد مارو بگیره یارو ترس میکن...      FEAR
6123  همون هارو مگه آهنگ جدیدای خوانندههای دهه پنجاه...  SURPRISE
6124                             نیم دگیرش چطور حل نیشد     OTHER

[6125 rows x 2 columns]


In [71]:
df

,text,label
0,خیلی کوچیک هستن و سایزشون بدرد نمیخوره میخوام ...,SAD
1,از صدای پرنده دم دمای صبح متنفرم متنفرم متنفرم,HATE
2,کیفیتش خیلی خوبه با شک خریدم ولی واقعا راضیم ...,SAD
3,چون همش با دوربین ثبتشده ایا میشه اعتراض زد و...,OTHER
4,این وضع ب طرز خندهداری گریه داره,SAD
...,...,...
6120,مرحوم پیشبینی آبکی زیاد میکرد مرحوم عجب آینده ...,SURPRISE
6121,کلا عین اعتقادات و توئیت زدناتون در قبال ران...,ANGRY
6122,خب وقتی میگی کسی بیاد مارو بگیره یارو ترس میکن...,FEAR
6123,همون هارو مگه آهنگ جدیدای خوانندههای دهه پنجاه...,SURPRISE


In [72]:
df_test

,text,label
0,این شاید اولین عزای عمومی واقعی است که یاد دار...,SAD
1,دیشب بعد از ارسال تویت مربوط به آثار باستانی ت...,HAPPY
2,کدوم شعبه پول نداده بگو الان برات آمار دقیق ب...,OTHER
3,امروز وسط یه بحث با بابا مامانم گفتم آدم باید ...,HAPPY
4,امشب گفت نامزدی دوستش که ادم روشنفکری است بهم ...,SAD
...,...,...
1146,یعنی این آهنگ مرغ سحر جوری ساختهشدهو روی این ش...,HAPPY
1147,درود بر همه ایرانیان شریف که در این سرما در تظ...,HAPPY
1148,امروز تولدم است عید است ولی شاد نیستم عید و ...,SAD
1149,لعنت به اونی که دلتنگ نگهت میداره,SAD


In [73]:
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])
df_test['label'] = label_encoder.transform(df_test['label'])

In [74]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("michellejieli/emotion_text_classifier")

tokenizer_config.json:   0%|          | 0.00/413 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [75]:
dataset = Dataset.from_pandas(df)
dataset_test = Dataset.from_pandas(df_test)

In [76]:
dataset[0]

{'text': 'خیلی کوچیک هستن و سایزشون بدرد نمیخوره میخوام پس بدم', 'label': 5}

In [77]:
dataset[0]

{'text': 'خیلی کوچیک هستن و سایزشون بدرد نمیخوره میخوام پس بدم', 'label': 5}

In [78]:
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 6125
})

In [79]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True , padding=True)

In [80]:
dataset_yok = dataset.map(preprocess_function, batched=True)
dataset_test_yok = dataset_test.map(preprocess_function, batched=True)


Map:   0%|          | 0/6125 [00:00<?, ? examples/s]

Map:   0%|          | 0/1151 [00:00<?, ? examples/s]

In [81]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [82]:
import evaluate

accuracy = evaluate.load("accuracy")

In [83]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [84]:
print("Label encoder classes:", label_encoder.classes_)

Label encoder classes: ['ANGRY' 'FEAR' 'HAPPY' 'HATE' 'OTHER' 'SAD' 'SURPRISE']


In [85]:
id2label = {
    0: "ANGRY",
    1: "FEAR",
    2: "HAPPY",
    3: "HATE",
    4: "OTHER",
    5: "SAD",
    6: "SURPRISE"
}

label2id = {label: id for id, label in id2label.items()}

In [86]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "michellejieli/emotion_text_classifier", num_labels=7, id2label=id2label, label2id=label2id
)

config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

In [87]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_yok,
    eval_dataset=dataset_test_yok,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,2.030387,0.199826
2,1.776500,1.848941,0.291051
3,1.388800,1.831536,0.315378


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,2.030387,0.199826
2,1.776500,1.848941,0.291051
3,1.388800,1.831536,0.315378
4,1.258600,1.748686,0.349262
5,1.258600,1.726774,0.363162
6,1.188500,1.765918,0.357950
7,1.153300,1.787588,0.376195
8,1.101900,1.742642,0.374457
9,1.101900,1.801584,0.392702
10,1.087300,1.781250,0.382276


TrainOutput(global_step=3830, training_loss=1.259619897588426, metrics={'train_runtime': 3137.1453, 'train_samples_per_second': 19.524, 'train_steps_per_second': 1.221, 'total_flos': 8114028724804416.0, 'train_loss': 1.259619897588426, 'epoch': 10.0})